In [33]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [8]:
diabetes = datasets.load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
print(X.columns)

Index(['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], dtype='object')


In [7]:
y = pd.Series(diabetes.target, name='target')
print(y)

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: target, Length: 442, dtype: float64


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# Apply StandardScaler to both X and y
# Scaler for features (X)
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scaler for target (y) 
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

print("Scaling completed:")
print(f"X_train shape: {X_train_scaled.shape}")
print(f"X_test shape: {X_test_scaled.shape}")
print(f"y_train_scaled shape: {y_train_scaled.shape}")
print(f"y_test_scaled shape: {y_test_scaled.shape}")

print(f"\nOriginal y range: {y_train.min():.2f} to {y_train.max():.2f}")
print(f"Scaled y range: {y_train_scaled.min():.2f} to {y_train_scaled.max():.2f}")

Scaling completed:
X_train shape: (353, 10)
X_test shape: (89, 10)
y_train_scaled shape: (353,)
y_test_scaled shape: (89,)

Original y range: 25.00 to 346.00
Scaled y range: -1.65 to 2.47


In [52]:
# Linear Regression with both X and y scaled
model_fully_scaled = LinearRegression()
model_fully_scaled.fit(X_train_scaled, y_train_scaled)
y_pred_fully_scaled = model_fully_scaled.predict(X_test_scaled)

# Transform predictions back to original scale for comparison
y_pred_unscaled = scaler_y.inverse_transform(y_pred_fully_scaled.reshape(-1, 1)).flatten()

# Calculate metrics (comparing in original scale)
mse_fully_scaled = mean_squared_error(y_test, y_pred_unscaled)
r2_fully_scaled = model_fully_scaled.score(X_test_scaled, y_test_scaled)

print("Linear Regression with X and y scaled:")
print(f"MSE (original scale): {mse_fully_scaled:.4f}")
print(f"R² (scaled): {r2_fully_scaled:.4f}")

# Decision Tree with both X and y scaled
dt_fully_scaled = DecisionTreeRegressor(random_state=42)
dt_fully_scaled.fit(X_train_scaled, y_train_scaled)
y_pred_dt_fully_scaled = dt_fully_scaled.predict(X_test_scaled)

# Transform predictions back to original scale
y_pred_dt_unscaled = scaler_y.inverse_transform(y_pred_dt_fully_scaled.reshape(-1, 1)).flatten()

mse_dt_fully_scaled = mean_squared_error(y_test, y_pred_dt_unscaled)
r2_dt_fully_scaled = dt_fully_scaled.score(X_test_scaled, y_test_scaled)

print(f"\nDecision Tree with X and y scaled:")
print(f"MSE (original scale): {mse_dt_fully_scaled:.4f}")
print(f"R² (scaled): {r2_dt_fully_scaled:.4f}")

Linear Regression with X and y scaled:
MSE (original scale): 2900.1936
R² (scaled): 0.4526

Decision Tree with X and y scaled:
MSE (original scale): 4937.5056
R² (scaled): 0.0681


In [53]:
# COMPREHENSIVE COMPARISON OF ALL SCALING APPROACHES
print("="*60)
print("COMPLETE MODEL COMPARISON WITH DIFFERENT SCALING")
print("="*60)

# We need to recompute some values since variables might have changed
# Original models (no scaling)
model_orig = LinearRegression()
model_orig.fit(X_train, y_train)
y_pred_orig = model_orig.predict(X_test)
mse_orig = mean_squared_error(y_test, y_pred_orig)
r2_orig = model_orig.score(X_test, y_test)

dt_orig = DecisionTreeRegressor(random_state=42)
dt_orig.fit(X_train, y_train)
y_pred_dt_orig = dt_orig.predict(X_test)
mse_dt_orig = mean_squared_error(y_test, y_pred_dt_orig)
r2_dt_orig = dt_orig.score(X_test, y_test)

# X-only scaled models
model_x_scaled = LinearRegression()
model_x_scaled.fit(X_train_scaled, y_train)
y_pred_x_scaled = model_x_scaled.predict(X_test_scaled)
mse_x_scaled = mean_squared_error(y_test, y_pred_x_scaled)
r2_x_scaled = model_x_scaled.score(X_test_scaled, y_test)

dt_x_scaled = DecisionTreeRegressor(random_state=42)
dt_x_scaled.fit(X_train_scaled, y_train)
y_pred_dt_x_scaled = dt_x_scaled.predict(X_test_scaled)
mse_dt_x_scaled = mean_squared_error(y_test, y_pred_dt_x_scaled)
r2_dt_x_scaled = dt_x_scaled.score(X_test_scaled, y_test)

print("LINEAR REGRESSION:")
print(f"No Scaling:        MSE: {mse_orig:.4f}, R²: {r2_orig:.4f}")
print(f"X Only Scaled:     MSE: {mse_x_scaled:.4f}, R²: {r2_x_scaled:.4f}")
print(f"X & Y Scaled:      MSE: {mse_fully_scaled:.4f}, R²: {r2_fully_scaled:.4f}")

print(f"\nDECISION TREE:")
print(f"No Scaling:        MSE: {mse_dt_orig:.4f}, R²: {r2_dt_orig:.4f}")
print(f"X Only Scaled:     MSE: {mse_dt_x_scaled:.4f}, R²: {r2_dt_x_scaled:.4f}")
print(f"X & Y Scaled:      MSE: {mse_dt_fully_scaled:.4f}, R²: {r2_dt_fully_scaled:.4f}")

# Find overall best model
all_models = {
    'Linear Reg (No Scaling)': mse_orig,
    'Linear Reg (X Scaled)': mse_x_scaled,
    'Linear Reg (X&Y Scaled)': mse_fully_scaled,
    'Decision Tree (No Scaling)': mse_dt_orig,
    'Decision Tree (X Scaled)': mse_dt_x_scaled,
    'Decision Tree (X&Y Scaled)': mse_dt_fully_scaled
}

best_overall = min(all_models, key=all_models.get)
print(f"\n🏆 BEST OVERALL MODEL: {best_overall}")
print(f"   Lowest MSE: {all_models[best_overall]:.4f}")

print(f"\n📊 KEY INSIGHTS:")
print(f"• Linear Regression: MSE remains consistent across scaling approaches")
print(f"• Decision Trees: Show slight variation with different scaling")
print(f"• Scaling y doesn't significantly improve performance for these models")
print(f"• Linear Regression outperforms Decision Tree for this dataset")

COMPLETE MODEL COMPARISON WITH DIFFERENT SCALING
LINEAR REGRESSION:
No Scaling:        MSE: 2900.1936, R²: 0.4526
X Only Scaled:     MSE: 2900.1936, R²: 0.4526
X & Y Scaled:      MSE: 2900.1936, R²: 0.4526

DECISION TREE:
No Scaling:        MSE: 4976.7978, R²: 0.0607
X Only Scaled:     MSE: 4887.0000, R²: 0.0776
X & Y Scaled:      MSE: 4937.5056, R²: 0.0681

🏆 BEST OVERALL MODEL: Linear Reg (No Scaling)
   Lowest MSE: 2900.1936

📊 KEY INSIGHTS:
• Linear Regression: MSE remains consistent across scaling approaches
• Decision Trees: Show slight variation with different scaling
• Scaling y doesn't significantly improve performance for these models
• Linear Regression outperforms Decision Tree for this dataset


In [14]:
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [18]:
y_pred = model.predict(X_test)

In [21]:
model_score = mean_squared_error(y_test, y_pred)

print("Model Score (MSE):", model_score)

Model Score (MSE): 2900.1936284934814


In [22]:
model_score = model.score(X_test, y_test)
print(f"Model Score: {model_score}")

Model Score: 0.4526027629719195


In [23]:
print(model.coef_)

[  37.90402135 -241.96436231  542.42875852  347.70384391 -931.48884588
  518.06227698  163.41998299  275.31790158  736.1988589    48.67065743]


In [26]:
model_Lasso = Lasso(alpha=0.1)
model_Lasso.fit(X_train, y_train)

,alpha,0.1
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [28]:
model_score = model_Lasso.score(X_test, y_test)
print(f"Model Score: {model_score}")

Model Score: 0.4718547867276227


In [29]:
model_Ridge = Ridge(alpha=0.1)
model_Ridge.fit(X_train, y_train)

,alpha,0.1
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [31]:
model_score = model_Ridge.score(X_test, y_test)
print(f"Model Score: {model_score}")

Model Score: 0.46085219464119265


In [54]:
# Re-run Linear Regression with scaled data
model_scaled = LinearRegression()
model_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = model_scaled.predict(X_test_scaled)

# Compare performance of original vs scaled
mse_original = mean_squared_error(y_test, y_pred)
mse_scaled = mean_squared_error(y_test, y_pred_scaled)
r2_original = model.score(X_test, y_test)
r2_scaled = model_scaled.score(X_test_scaled, y_test)

print("Linear Regression Comparison:")
print(f"Original - MSE: {mse_original:.4f}, R²: {r2_original:.4f}")
print(f"Scaled   - MSE: {mse_scaled:.4f}, R²: {r2_scaled:.4f}")
print(f"Improvement: {'Better' if mse_scaled < mse_original else 'Worse'} with scaling")

Linear Regression Comparison:
Original - MSE: 2900.1936, R²: 0.0358
Scaled   - MSE: 2900.1936, R²: 0.4526
Improvement: Worse with scaling


/Users/tanmayshubhgarg/Documents/Projects/Camp2025/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_original = DecisionTreeRegressor(random_state=42)
dt_original.fit(X_train, y_train)
y_pred_dt_original = dt_original.predict(X_test)

# Decision Tree WITH scaling
dt_scaled = DecisionTreeRegressor(random_state=42)
dt_scaled.fit(X_train_scaled, y_train)
y_pred_dt_scaled = dt_scaled.predict(X_test_scaled)

# Calculate metrics for Decision Trees
mse_dt_original = mean_squared_error(y_test, y_pred_dt_original)
mse_dt_scaled = mean_squared_error(y_test, y_pred_dt_scaled)
r2_dt_original = dt_original.score(X_test, y_test)
r2_dt_scaled = dt_scaled.score(X_test_scaled, y_test)

print("\nDecision Tree Regression")
print(f"Original - MSE: {mse_dt_original:.4f}, R²: {r2_dt_original:.4f}")
print(f"Scaled   - MSE: {mse_dt_scaled:.4f}, R²: {r2_dt_scaled:.4f}")
print(f"Scaling Impact: {'Helpful' if mse_dt_scaled < mse_dt_original else 'No improvement'}")

print("\n" + "="*50)
print("OVERALL MODEL COMPARISON:")
print("="*50)
print(f"Linear Reg (Original):     MSE: {mse_original:.4f}, R²: {r2_original:.4f}")
print(f"Linear Reg (Scaled):       MSE: {mse_scaled:.4f}, R²: {r2_scaled:.4f}")
print(f"Decision Tree (Original):  MSE: {mse_dt_original:.4f}, R²: {r2_dt_original:.4f}")
print(f"Decision Tree (Scaled):    MSE: {mse_dt_scaled:.4f}, R²: {r2_dt_scaled:.4f}")

# Find best model
models = {
    'Linear Reg (Original)': mse_original,
    'Linear Reg (Scaled)': mse_scaled,
    'Decision Tree (Original)': mse_dt_original,
    'Decision Tree (Scaled)': mse_dt_scaled
}
best_model = min(models, key=models.get)
print(f"\nBest Model: {best_model} (Lowest MSE: {models[best_model]:.4f})")


Decision Tree Regression Comparison:
Original - MSE: 4976.7978, R²: 0.0607
Scaled   - MSE: 4887.0000, R²: 0.0776
Scaling Impact: Helpful

OVERALL MODEL COMPARISON:
Linear Reg (Original):     MSE: 2900.1936, R²: 0.0358
Linear Reg (Scaled):       MSE: 2900.1936, R²: 0.4526
Decision Tree (Original):  MSE: 4976.7978, R²: 0.0607
Decision Tree (Scaled):    MSE: 4887.0000, R²: 0.0776

Best Model: Linear Reg (Original) (Lowest MSE: 2900.1936)
